In [1]:
# import libraries
import sys
sys.path.append("/Users/SJp/Documents/project_local/VIB-LLM-SS/ml-summerschool-2025/topic-1_data-integration-and-llms/project")

import pandas as pd
import numpy as np
import importlib
import src.VIB_project_functions as FF
importlib.reload(FF)
import warnings
warnings.filterwarnings('ignore')

from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage, BaseMessage
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from typing import Optional, List, Union
from pydantic import BaseModel, Field, field_validator
from typing import List, Optional
from langchain_core.output_parsers import StrOutputParser
from langchain_core.output_parsers import PydanticOutputParser
import json
import os
import re
import glob
import json
from json import JSONEncoder


In [2]:
global_path  = "/Users/SJp/Documents/project_local/VIB-LLM-SS/ml-summerschool-2025/topic-1_data-integration-and-llms/project"
# Load data
# cytokines = pd.read_csv(f"{global_path}/data/Cytokines.csv", header=0, index_col=0).transpose()
# clinical = pd.read_csv(f"{global_path}/data/Clinical.csv", header=0, index_col=0).transpose()
# proteomics = pd.read_csv(f"{global_path}/data/MyelomaCells_proteomics.csv", header=0, index_col=0).transpose()
# drugresponse = pd.read_csv(f"{global_path}/data/DrugResponse.csv", header=0, index_col=0).transpose().dropna()
 
# # make sure all the above dataframes have the same patients in the same order
# common_patients = set(clinical.index) & set(cytokines.index) & set(proteomics.index) & set(drugresponse.index)
# common_patients = sorted(list(common_patients))
# clinical = clinical.loc[common_patients].sort_index()
# cytokines = cytokines.loc[common_patients].sort_index()
# proteomics = proteomics.loc[common_patients].sort_index()
# drugresponse = drugresponse.loc[common_patients].sort_index()
# clinical_numeric = pd.get_dummies(clinical)

# drug_classes = pd.read_csv(f"{global_path}/data/drugresponse_classes.csv", index_col=0)

## ML model to predict the drug response

In [3]:
# results, patient_explanations=FF.run_drug_classification(
#     proteomics, cytokines, clinical_numeric, drug_classes, 
#     output_dir=f"{global_path}/results")

In [4]:
#FF.save_patient_graphs_json(patient_explanations, results, clinical, outdir=f"{global_path}/results/patient_graphs_json")

## LLM empowered results integration

### Generate the summary report of drug response prediction from ML model

* Reading patient JSON

In [5]:
from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage, BaseMessage
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from typing import Optional, List, Union
from pydantic import BaseModel, Field, field_validator
from typing import List, Optional
from langchain_core.output_parsers import StrOutputParser
from langchain_core.output_parsers import PydanticOutputParser
import json
import os
import re
import glob
import json
from json import JSONEncoder


In [6]:
# Pick via provider:model string → works across providers
llm = init_chat_model(model="gemini-2.5-flash",
                      model_provider="google_genai",
                      temperature=0.2)


prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a biomedical-AI assistant that interprets predictions from a AI-powered predictive model for clinicians.
You are given a JSON with some information regarding the patient and 2 drugs. In the JSON, you are given:
- Patient ID
- Disease type
- For each drug, you will be given:
    - The drug name
    - The predicted class. This class can be one of three options: 
        - No effect if the drug is predicted as having no effect on treating the patient disease
        - Positive response if the drug is predicted as having a positive effect on treating the patient disease
        - Adverse effects if the drug is predicted as having a negative effect on the patient disease
    - Each predicted class has an associated probability
    - Each predicted class has associated features, that are responsible for the prediction. To reflect the importance of these features on the prediction we have the SHAP values. We have the top positive SHAP values and the top negative SHAP values.

Taking into account this JSON and the information explained above, I want you as a smart biomedical-AI assistant to pick the best of the two drugs. I don't want you to talk about the accuracy of the predicition for more than one sentence.
Once you have  picked the best drug for my patient, I want you to write a small report on the chosen drug, please include both positive an negative points about the drug, make it as straigthforward as possible (a maximum of 10 bullet points in total), and targeted towards clinicians.
After that, please write a short paragraph about the features involved in the decision making process, and look in the litterature for information about the relationship between these features and the disease the patient has."""),
    ("human", "{JSON_input}")
])

In [7]:

# Path to your JSON file (e.g., patient PKG or classification output)
json_path = f"{global_path}/results/MM082.json"

with open(json_path, "r") as f:
    patient_json = json.load(f)

patient_id = "MM082"

In [8]:
chain = prompt | llm | StrOutputParser()
response = chain.invoke({"JSON_input": patient_json})
print(response)

class model_prediction_report(BaseModel):
    patient_ID: Optional[Union[str, int]]  # "555-1234", 5551234, or None
    disease_type: str = Field(description="e.g., Melanoma")
    decision_making_process: str 

parser = PydanticOutputParser(pydantic_object=model_prediction_report)
format_instructions = parser.get_format_instructions()
prompt = ChatPromptTemplate.from_messages([
    ("system", "Extract per schema:\n{format_instructions}"),
    ("human", "{text}"),
]).partial(format_instructions=format_instructions)

parsing_llm = prompt | llm | parser

# if `drug_text` is an AIMessage, use .content; otherwise pass the raw string
result1 = parsing_llm.invoke({"text": response})
print(result1)

# class MyEncoder(JSONEncoder):
#     def default(self, o):
#         return o.__dict__
# MyEncoder().encode(result)

# with open(".json", "w") as f:
#     json.dump(MyEncoder().encode(result), f, indent=2)

ChatGoogleGenerativeAIError: Invalid argument provided to Gemini: 400 * GenerateContentRequest.contents: contents is not specified


### Functional Analysis of SHAP Features by Predicted Drug Response Class

In [ ]:
# notebook.ipynb
import json
from src.GSEA_bioassistant import get_llm_with_tools, get_prompt_chain, analyze_patient, save_patient_summary_html
# --- Setup LLM + chain ---
llm_with_tools = get_llm_with_tools()
chain = get_prompt_chain(llm_with_tools)

# --- Analyze patient ---
results, summaries = analyze_patient(patient_json, patient_id, chain)

# --- Show results ---
for cls, summary in summaries.items():
    print(f"\n### Drugs Predicted as {cls}")
    print(summary)

/Users/SJp/Documents/project_local/VIB-LLM-SS/ml-summerschool-2025/topic-1_data-integration-and-llms/project/src/GSEA_bioassistant.py:90: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use :meth:`~invoke` instead.
  enrichr_query({"gene_list": list(set(feats["positive"]))}) if feats["positive"] else None



### Drugs Predicted as no_effect
The positive SHAP features, which support a "no_effect" prediction, are primarily associated with general cellular regulatory processes and T-cell biology. These include the regulation of cell-cell adhesion (integrin-mediated), posttranscriptional regulation of gene expression, and protein-containing complex assembly. Additionally, several terms point to the regulation of immune responses, specifically the positive regulation of T cell proliferation and activation. There's also a mention of substantia nigra development and cellular response to indole-3-methanol.

Conversely, the negative SHAP features, which argue against a "no_effect" prediction, strongly indicate active and specific immune responses. These features highlight neutrophil-mediated immunity, including neutrophil degranulation and activation. They also emphasize the positive regulation of T cell mediated cytotoxicity and immunity, as well as the interferon-gamma-mediated signaling pathway

### **Open FDA Drug Information**

In [ ]:
import requests, urllib.parse

def get_openfda_label(ingredient):
    # Standardize ingredient name
    ingredient = urllib.parse.quote(ingredient)
    print(ingredient)
    base = "https://api.fda.gov/drug/label.json"
    q = f'openfda.substance_name:"{ingredient}"'
    r = requests.get(base, params={"search": q, "limit": 1})
    r.raise_for_status()
    res = r.json().get("results", [])
    if not res:
        q2 = f'openfda.brand_name:"{ingredient}"'
        r = requests.get(base, params={"search": q2, "limit": 1})
        r.raise_for_status()
        res = r.json().get("results", [])
        if not res:
            return None
    return res[0]

item = get_openfda_label("ibuprofen")

if item:
    for key in ("purpose","indications_and_usage", "adverse_reactions", "warnings", "dosage_and_administration"):
        if key in item:
            print(f"\n=== {key} ===\n{item[key][0][:800]}")
else:
    print("Cannot find record on openFDA for that name.")

ibuprofen

=== purpose ===
Purpose Pain reliever/fever reducer

=== indications_and_usage ===
Uses temporarily relieves minor aches and pains due to: headache toothache backache menstrual cramps the common cold muscular aches minor pain of arthritis temporarily reduces fever

=== warnings ===
Warnings Allergy alert: Ibuprofen may cause a severe allergic reaction, especially in people allergic to aspirin. Symptoms may include: rash facial swelling asthma (wheezing) hives skin reddening shock blisters If an allergic reaction occurs, stop use and seek medical help right away. Stomach bleeding warning: This product contains an NSAID, which may cause severe stomach bleeding. The chance is higher if you: take more or for a longer time than directed take a blood thinning (anticoagulant) or steroid drug have had stomach ulcers or bleeding problems have 3 or more alcoholic drinks every day while using this product are age 60 or older take other drugs containing prescription or nonprescription N

### Combine results in one JSON

In [ ]:
# --- Combine everything into one JSON and save ---
from datetime import datetime

def _as_dict(x):
    """Make objects JSON-serializable."""
    if x is None:
        return None
    if hasattr(x, "model_dump"):
        return x.model_dump()
    if hasattr(x, "dict"):
        return x.dict()
    if hasattr(x, "to_dict"):  # pandas DataFrame, Series, etc.
        return x.to_dict()
    if isinstance(x, (list, tuple)):
        return [_as_dict(i) for i in x]
    if isinstance(x, dict):
        return {k: _as_dict(v) for k, v in x.items()}
    return x  # assume it's a primitive type


combined = {
    "patient_id": patient_id,
    "disease_type": (
        getattr(result1, "disease_type", None)
        or patient_json.get("disease_type")
    ),
    "models": {
        "chat_model": "gemini-2.5-flash",
        "temperature": 0.2,
    },
    "assistant_free_text_report": (
        response.content if hasattr(response, "content") else str(response)
    ),
    "assistant_structured_report": _as_dict(result1),
    "prediction_pipeline": {   # handles DataFrames
        "summaries": _as_dict(summaries),  # handles dict of strings / dfs
    },
}


# Save next to your other outputs
os.makedirs(os.path.join(global_path, "combined"), exist_ok=True)
out_path = os.path.join(global_path, "combined", f"{patient_id}_combined.json")

with open(out_path, "w", encoding="utf-8") as f:
    json.dump(combined, f, indent=2, ensure_ascii=False)


print(f"Wrote combined JSON → {out_path}")


Wrote combined JSON → /Users/SJp/Documents/project_local/VIB-LLM-SS/ml-summerschool-2025/topic-1_data-integration-and-llms/project/combined/MM082_combined.json


In [ ]:
def json_to_html(data: dict) -> str:
    """Convert summary JSON (supports multiple schema versions) into styled HTML."""

    patient_id = _extract_field(
        data, ["patient_ID", "patient_id", ("assistant_structured_report", "patient_ID")], "Unknown"
    )
    disease = _extract_field(
        data, ["disease_type", ("assistant_structured_report", "disease_type")], "Unknown"
    )
    drug = _extract_field(
        data,
        ["recomended_drug_name", "recommended_drug_name", ("assistant_structured_report", "recomended_drug_name")],
        "N/A",
    )
    drug_info = _extract_field(
        data,
        ["info_on_recommended_drug", ("assistant_structured_report", "info_on_recommended_drug"), "assistant_free_text_report"],
        "",
    )
    decision = _extract_field(
        data,
        ["decision_making_process", ("assistant_structured_report", "decision_making_process"), "assistant_free_text_report"],
        "",
    )

    # --- HTML ---
    html_parts = [
        "<html><head><meta charset='utf-8'>",
        "<style>",
        "body { font-family: 'Segoe UI', Tahoma, sans-serif; margin: 40px; background: #f9f9f9; color: #2c3e50; }",
        "header { background: #004080; color: white; padding: 20px; border-radius: 8px; }",
        "header h1 { margin: 0; font-size: 26px; }",
        "header p { margin: 5px 0; font-size: 16px; }",
        "section { background: white; padding: 20px; margin-top: 20px; border-radius: 8px; box-shadow: 0 2px 5px rgba(0,0,0,0.1); }",
        "h2 { color: #004080; margin-top: 0; border-bottom: 2px solid #004080; padding-bottom: 6px; }",
        "h3 { color: #16a085; margin-top: 18px; }",
        "p, li { font-size: 15px; line-height: 1.6; }",
        "ul { margin: 0.25rem 0 0.75rem 1.25rem; }",
        "pre { background: #f4f6f7; padding: 12px; border-left: 4px solid #004080; border-radius: 4px; white-space: pre-wrap; font-size: 14px; }",
        "</style></head><body>",

        "<header>",
        "<h1>Patient Drug Response Prediction Report</h1>",
        f"<p><strong>Patient ID:</strong> {patient_id}</p>",
        f"<p><strong>Disease:</strong> {disease}</p>",
        "</header>",

        "<section>",
        "<h2>Recommended Treatment</h2>",
        f"<p><strong>Drug Regimen:</strong> {drug}</p>",
        "</section>",

        "<section>",
        "<h2>Clinical Considerations</h2>",
        format_clinical_text(drug_info),
        "</section>",

        "<section>",
        "<h2>Decision Rationale</h2>",
        format_decision_text(decision),
        "</section>",

        "</body></html>",
    ]
    return "\n".join(html_parts)


In [ ]:
import json

Outpath = "/Users/SJp/Documents/project_local/VIB-LLM-SS/ml-summerschool-2025/topic-1_data-integration-and-llms/project/combined/MM082_combined.json"

# Load JSON (fix double-encoded JSON if needed)
with open(Outpath, "r") as f:
    patient_json = json.load(f)
if isinstance(patient_json, str):
    import json as _json
    patient_json = _json.loads(patient_json)


# Save to file
json_to_html(patient_json, "reports/MM082_report.html")

TypeError: json_to_html() takes 1 positional argument but 2 were given